# R-Vg Measurement
This example notebook shows how to measure resistance (using SR830 lock-in amplifier) as a function of gate voltage (using Keithley 2400). This is an example to show how to sweep one instrument and measure two instrument at the same time.

In [ ]:
import sys
# add path to GrapheneLab-Measurement-Code folder 
sys.path.append(r"C:\Users\Username\Documents\GrapheneLab-Measurement-Code")

In [ ]:
import os
import numpy as np

# import pyqtgraph for live plotting
import pyqtgraph as pg
import pyqtgraph.multiprocess as mp

# import instrument drivers
from instrument.standford_research_systems.sr830 import SR830
from instrument.keithley.k6430 import K6430
from instrument.keithley.k2400 import K2400
from instrument.keithley.k2182 import K2182

# import measurement method
from measurement.do_device_sweep import do_device_sweep

In [ ]:
# set up live plotting
pg.mkQApp()
graph_proc = mp.QtProcess()
rpg = graph_proc._import('pyqtgraph')

In [ ]:
# assign instrument address
dev_source = K6430(24)
lockin1 = SR830(8)

# initialize instruments
dev_source.initialize(
    mode='VOLT', source_range=0.2, sense_range=1e-6, compliance=1e-6, 
    ramp_step=10e-3, auto_sense_range=False
)

lockin1.initialize(auto_range=False) # auto range function doesn't very well. Recommend to disable it.

# define instrument to sweep and read
sweep_inst = dev_source
read_inst = [dev_source, lockin1]

# if the devices are connected succesfully, you should see device name below

In [ ]:
# put measurement details in filename
device_name = "device name"
meas_type = "measurement type"
junction_name = "junction name"
parameter = "measurement parameters"
condition = "measurement condition"
seperator = '_'

# file name will be "device name_measurement type_junction name_measurement parameters_measurement condition"
# you can customise the file name according to your need. Any string is OK
filename = seperator.join([device_name, meas_type, junction_name, parameter, condition])

# you can also put comment inside the file
comment_string = "comment"

In [ ]:
do_device_sweep(
    # don't touch these lines. They are used to set up measurement windows, etc.
    graph_proc, rpg, file_title,
    comment=comment_string,
    
    # magnetic field setting, in Tesla
    b_set=0., persist=True, # persit mode
    ignore_magnet=True, # ignore magnetic field control if you are not using it
    
    # temperature setting, in Kelvin
    t_set=-1, # put -1 if you don't want temperature control
    
    # pass sweep and read instruments which are defined in cells above
    sweep_inst, read_inst,
    
    # you can set a fixed value to another instrument while sweeping the sweep instrument
    set_inst=[], set_value=[], finish_value=[], pre_value=[], 
    
    # sweeping parameters
    sweep_start=-20e-3,sweep_stop=20e-3, sweep_mid=[], sweep_step=0.2e-3, sweep_finish=0.0,

    delay=0.1, # # delay between each measurement point, in seconds
    timeout=10, # maximum waiting time for the system to reach b_set and t_set, in minutes
    wait=0.1, # waiting time after b_set and t_set is reached before measurement start, in minutes
)